In [ ]:
#HSV
import cv2
def empty(v):
    pass

block = cv2.imread('./images/img5.png')
block = cv2.resize(block, (0, 0), fx=.7, fy=.7)

cv2.namedWindow('TrackBar')
cv2.resizeWindow('TrackBar', 640, 320)

cv2.createTrackbar('Hue Min', 'TrackBar', 0, 179, empty)
cv2.createTrackbar('Hue Max', 'TrackBar', 179, 179, empty)
cv2.createTrackbar('Sat Min', 'TrackBar', 0, 255, empty)
cv2.createTrackbar('Sat Max', 'TrackBar', 255, 255, empty)
cv2.createTrackbar('Val Min', 'TrackBar', 0, 255, empty)
cv2.createTrackbar('Val Max', 'TrackBar', 255, 255, empty)

hsv = cv2.cvtColor(block,cv2.COLOR_BGR2HSV)
while True:
    h_min = cv2.getTrackbarPos('Hue Min', 'TrackBar')
    h_max = cv2.getTrackbarPos('Hue Max', 'TrackBar')
    s_min = cv2.getTrackbarPos('Sat Min', 'TrackBar')
    s_max = cv2.getTrackbarPos('Sat Max', 'TrackBar')
    v_min = cv2.getTrackbarPos('Val Min', 'TrackBar')
    v_max = cv2.getTrackbarPos('Val Max', 'TrackBar')
    # print(h_min, h_max, s_min, s_max, v_min, v_max)
    
    lower = np.array([h_min, s_min, v_min])
    upper = np.array([h_max, s_max, v_max])
    
    mask = cv2.inRange(hsv, lower, upper)
    result = cv2.bitwise_and(block, block, mask=mask)

    cv2.imshow('block', block)
    # cv2.imshow('hsv', hsv)
    cv2.imshow('mask', mask)
    cv2.imshow('result', result)
    

    key = cv2.waitKey(1)
    if key == 27:
        break

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import numpy as np
from scipy.spatial.transform import Rotation as R

def transform_matrix(x, y, z, rx, ry, rz):
    # 轉換為弧度
    rx, ry, rz = np.radians(rx), np.radians(ry), np.radians(rz)

    # 計算旋轉矩陣
    rotation_matrix = R.from_euler('xyz', [rx, ry, rz], degrees=False).as_matrix()

    # 構建轉移矩陣
    transformation_matrix = np.vstack([np.hstack([rotation_matrix, np.array([[x], [y], [z]])]),
                                      np.array([0, 0, 0, 1])])

    return transformation_matrix


In [ ]:
import numpy as np

def rotation_x(theta):
    return np.array([[1, 0, 0],
                     [0, np.cos(theta), -np.sin(theta)],
                     [0, np.sin(theta), np.cos(theta)]])

def rotation_y(theta):
    return np.array([[np.cos(theta), 0, np.sin(theta)],
                     [0, 1, 0],
                     [-np.sin(theta), 0, np.cos(theta)]])

def rotation_z(theta):
    return np.array([[np.cos(theta), -np.sin(theta), 0],
                     [np.sin(theta), np.cos(theta), 0],
                     [0, 0, 1]])

def transform_matrix(x, y, z, rx, ry, rz):
    rx = np.radians(rx)
    ry = np.radians(ry)
    rz = np.radians(rz)

    rotation_matrix = np.dot(rotation_z(rz), np.dot(rotation_y(ry), rotation_x(rx)))
    translation_vector = np.array([[x], [y], [z]])

# 合併旋轉矩陣和平移向量
    transformation_matrix = np.vstack([np.hstack([rotation_matrix, translation_vector]),
                                      np.array([0, 0, 0, 1])])

    return transformation_matrix

In [ ]:
import numpy as np
from scipy.spatial.transform import Rotation as R


def calculate_transform_matrix(initial_position, final_position):
    # 初始座標和移動後的座標
    p1 = np.array(initial_position)
    p2 = np.array(final_position)

    # 計算平移向量
    translation_vector = p2 - p1

    # 計算旋轉矩陣
    direction = p2 - p1
    axis = np.cross(np.array([1, 0, 0]), direction)
    angle = np.arccos(np.dot(np.array([1, 0, 0]), direction) / (np.linalg.norm(direction)))

    rotation_matrix = R.from_rotvec(axis * angle).as_matrix()

    # 構建轉移矩陣
    transformation_matrix = np.vstack([np.hstack([rotation_matrix, translation_vector.reshape(3, 1)]),
                                      np.array([0, 0, 0, 1])])

    return transformation_matrix
np.linalg.inv(transform_matrix(200,500,509,-180, 0, -60)) @ calculate_transform_matrix([0,0,509],[200,500,509])